# 単体テスト

## 統計データ取得

In [1]:
import urllib.parse
import urllib.request
import json

def get_estat_response(params):
  # appId
  params['appId'] = '724e5b90772a3e9289f41a253e4e7e32438f4fff'
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(params)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [22]:
params = {
    'statsDataId': '0003445078',
    # 'cdCat01': 'A1101',
    }

res = get_estat_response(params)
print(res)

{'GET_STATS_DATA': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2023-01-11T15:27:37.175+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0003445078', 'DATA_FORMAT': 'J', 'START_POSITION': 1, 'METAGET_FLG': 'Y'}, 'STATISTICAL_DATA': {'RESULT_INF': {'TOTAL_NUMBER': 12258, 'FROM_NUMBER': 1, 'TO_NUMBER': 12258}, 'TABLE_INF': {'@id': '0003445078', 'STAT_NAME': {'@code': '00200521', '$': '国勢調査'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '令和２年国勢調査 人口等基本集計\u3000（主な内容：男女・年齢・配偶関係，世帯の構成，住居の状態，母子・父子世帯，国籍など）', 'TITLE': {'@no': '1-1-1', '$': '総人口・総世帯数・男女・年齢・配偶関係 男女別人口－全国，都道府県，市区町村（2000年（平成12年）市区町村含む）'}, 'CYCLE': '-', 'SURVEY_DATE': 202010, 'OPEN_DATE': '2021-11-30', 'SMALL_AREA': 0, 'MAIN_CATEGORY': {'@code': '02', '$': '人口・世帯'}, 'SUB_CATEGORY': {'@code': '01', '$': '人口'}, 'OVERALL_TOTAL_NUMBER': 12258, 'UPDATED_DATE': '2022-12-28', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '令和２年国勢調査', 'TABULATION_SUB_CATEGORY1': '人口等基本集計\u3000（主な内容：男女・年齢・配偶関係，世帯の構成，住居

### categories

In [3]:
import pandas as pd

def df_categories(res: dict) :
  CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  categories = list(filter(lambda x: x['@id'] == 'cat01', CLASS_OBJ))[0]['CLASS']
  
  # DataFrameに変換
  df = pd.DataFrame(categories) if type(categories) is list else pd.DataFrame([categories])

  # 列の整理
  columns = {'@code':'categoryCode','@name':'categoryName'}
  df = df[list(columns.keys())].rename(columns = columns)
  
  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  return df

In [23]:
df_categories(res)

,categoryCode,categoryName
0,0,総数
1,1,男
2,2,女


### areas

In [5]:
def df_areas(res: dict) :
  CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  areas = list(filter(lambda x: x['@id'] == 'area', CLASS_OBJ))[0]['CLASS']

  # DataFrameに変換
  df = pd.DataFrame(areas) if type(areas) is list else pd.DataFrame([areas])

  # 列の整理
  columns = {'@code':'areaCode','@name':'areaName'}
  df = df[list(columns.keys())].rename(columns = columns)

  return df

In [6]:
df_areas(res).head()

,areaCode,areaName
0,00000,全国
1,01000,北海道
2,02000,青森県
3,03000,岩手県
4,04000,宮城県


### times

In [7]:
def df_times(res: dict) -> list:
  CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  times = list(filter(lambda x: x['@id'] == 'time', CLASS_OBJ))[0]['CLASS']

  # DataFrameに変換
  df = pd.DataFrame(times) if type(times) is list else pd.DataFrame([times])

  # 列の整理
  columns = {'@code':'timeCode','@name':'timeName'}
  df = df[list(columns.keys())].rename(columns = columns)

  return df

In [8]:
df_times(res).head()

,timeCode,timeName
0,1975100000,1975年度
1,1976100000,1976年度
2,1977100000,1977年度
3,1978100000,1978年度
4,1979100000,1979年度


### formated Data

In [9]:
import pandas as pd

def df_data(res):
  VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  
  # DataFrameに変換
  df = pd.DataFrame(VALUE)

  # 欠損値の処理
  df = df.dropna().replace('-', '0').replace('X', '0')
  
  # 列の整理
  columns = {'@cat01':'categoryCode','@area':'areaCode','@time':'timeCode','$':'value','@unit':'unit'}
  df = df[list(columns.keys())].rename(columns = columns)
  
  # merge
  df = pd.merge(df, df_categories(res), on='categoryCode', how='left')
  df = pd.merge(df, df_areas(res), on='areaCode', how='left')
  df = pd.merge(df, df_times(res), on='timeCode', how='left')
  
  # 型変換
  if isinstance(float(df.iloc[1,3]), int):
    df = df.astype({'value': int})
  else:
    df = df.astype({'value': float})

  return df

In [20]:
params = {
    'statsDataId': '0003445078',
    # 'cdCat01': 'A1231',
    }

res = get_estat_response(params)
df_data(res).head()

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName
0,0,00000,2020000000,126146099.0,人,総数,全国,2020年
1,0,01000,2020000000,5224614.0,人,総数,北海道,2020年
2,0,01100,2020000000,1973395.0,人,総数,札幌市,2020年
3,0,01101,2020000000,248680.0,人,総数,札幌市中央区,2020年
4,0,01102,2020000000,289323.0,人,総数,札幌市北区,2020年


## メタ情報

In [11]:
import urllib.parse
import urllib.request
import json

def get_estat_meta(statsDataId):
  params ={
      'statsDataId': statsDataId,
      'appId': '724e5b90772a3e9289f41a253e4e7e32438f4fff',
  }

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(params)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [12]:
meta = get_estat_meta('0000010101')
print(meta)

{'GET_META_INFO': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2023-01-11T15:17:37.111+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0000010101', 'DATA_FORMAT': 'J'}, 'METADATA_INF': {'TABLE_INF': {'@id': '0000010101', 'STAT_NAME': {'@code': '00200502', '$': '社会・人口統計体系'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '都道府県データ 基礎データ', 'TITLE': {'@no': '0000010101', '$': 'Ａ\u3000人口・世帯'}, 'CYCLE': '年度次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2022-03-04', 'SMALL_AREA': 0, 'COLLECT_AREA': '全国', 'MAIN_CATEGORY': {'@code': '99', '$': 'その他'}, 'SUB_CATEGORY': {'@code': '99', '$': 'その他'}, 'OVERALL_TOTAL_NUMBER': 500976, 'UPDATED_DATE': '2022-03-04', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '都道府県データ', 'TABULATION_SUB_CATEGORY1': '基礎データ'}, 'DESCRIPTION': {'TABULATION_CATEGORY_EXPLANATION': '社会・人口統計体系の都道府県ごとに集計したデータを提供します。'}, 'TITLE_SPEC': {'TABLE_NAME': 'Ａ\u3000人口・世帯'}}, 'CLASS_INF': {'CLASS_OBJ': [{'@id': 'tab', '@name': '観測値', 'CLASS': {'@code': '00001', 

### カテゴリ情報

In [13]:
def catelgoryCodes(statsDataId):
  # メタ情報の取得
  meta= get_estat_meta(statsDataId)

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']
  cat01 = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']
  
  return [d.get('@code') for d in cat01]

In [14]:
print(catelgoryCodes('0000010101'))

['A1101', 'A110101', 'A110102', 'A1102', 'A110201', 'A110202', 'A1201', 'A120101', 'A120102', 'A1202', 'A120201', 'A120202', 'A1203', 'A120301', 'A120302', 'A1204', 'A120401', 'A120402', 'A1205', 'A120501', 'A120502', 'A1206', 'A120601', 'A120602', 'A1207', 'A120701', 'A120702', 'A1208', 'A120801', 'A120802', 'A1209', 'A120901', 'A120902', 'A1210', 'A121001', 'A121002', 'A1211', 'A121101', 'A121102', 'A1212', 'A121201', 'A121202', 'A1213', 'A121301', 'A121302', 'A1214', 'A121401', 'A121402', 'A1215', 'A121501', 'A121502', 'A1216', 'A121601', 'A121602', 'A1217', 'A121701', 'A121702', 'A1218', 'A121801', 'A121802', 'A1219', 'A121901', 'A121902', 'A1220', 'A122001', 'A122002', 'A1221', 'A122101', 'A122102', 'A1231', 'A1301', 'A130101', 'A130102', 'A1302', 'A130201', 'A130202', 'A1303', 'A130301', 'A130302', 'A1304', 'A130401', 'A130402', 'A1305', 'A130501', 'A130502', 'A1306', 'A130601', 'A130602', 'A1401', 'A140101', 'A140102', 'A1402', 'A140201', 'A140202', 'A1403', 'A140301', 'A140302'

## BigQueryへの登録

### ライブラリ

pandas-gbqではなくgoogle-cloud-bigqueryを使う

https://cloud.google.com/bigquery/docs/pandas-gbq-migration?hl=ja

In [ ]:
!pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'
!pip install pyarrow

### GCP認証

In [16]:
from google.colab import auth
auth.authenticate_user()

### プロジェクト設定

In [17]:
project_id='primal-buttress-342908'
dataset_id = 'estat_origin'

### GBQに保存する関数

In [18]:
from google.cloud import bigquery

def estat_to_gbq(params):
  # estatレスポンスの取得
  res = get_estat_response(params)
  
  # Dataframe作成  
  df = df_data(res)
  
  # table設定
  table_id = params['statsDataId'] + '_' + params['cdCat01']
  
  # スキーマ
  job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')

  # bigqueryへ保存
  client = bigquery.Client(project=project_id)
  job = client.load_table_from_dataframe(df, f'{dataset_id}.{table_id}', job_config=job_config).result()

  # 確認メッセージ  
  table = client.get_table(f'{dataset_id}.{table_id}') 
  print("{} 件のデータを「{}」に保存しました。".format(table.num_rows, table_id))
  
  return

In [19]:
params = {
    'statsDataId': '0000020201',
    'cdCat01': 'A1101',
    }

estat_to_gbq(params)

15313 件のデータを「0000020201_A1101」に保存しました。


# 一括処理

In [ ]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

def save_gbq(data):
  for statsDataId in data:
    for c in catelgoryCodes(statsDataId):
      params = {
          'statsDataId': statsDataId,
          'cdCat01': c,
      }

      client = bigquery.Client(project=project_id)
      table_id = params['statsDataId'] + '_' + params['cdCat01'] 
      
      try:
        client.get_table(f'{dataset_id}.{table_id}')  
        print("Table {} already exists.".format(table_id))
      except NotFound:
        estat_to_gbq(params)


## 社会・人口統計体系（都道府県）

https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00200502&tstat=000001111375&cycle=8&tclass1=000001111377&tclass2val=0

In [ ]:
# 統計表のリスト
data = ['0000010101', '0000010102', '0000010103', '0000010104', '0000010105', '0000010106', '0000010107', 
        '0000010108', '0000010109', '0000010110', '0000010111', '0000010112', '0000010113']

In [ ]:
save_gbq(data)

## 社会・人口統計体系（市区町村）

https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00200502&tstat=000001111376&cycle=8&tclass1=000001111380&collect_area=200&result_page=1&tclass2val=0

In [ ]:
# 統計表のリスト
data = ['0000020201', '0000020202', '0000020203', '0000020204', '0000020205', '0000020206', 
        '0000020207', '0000020208', '0000020209', '0000020210', '0000020211']
save_gbq(data)

# 国勢調査